# SQL III

_**Author**: Boom D. (DSI-NYC), Mahdi S. (DSI-NYC), Dan Wilhelm (LA)_
***

For this lesson, we will use the `hacker_news` public dataset on [Google BigQuery](https://cloud.google.com/bigquery/).

## Contents
- [Intermediate SQL I: Special Functions](#Advanced-SQL-I:-Special-Functions)
    - **I. String Manipulation**
        - [`UPPER()`](#UPPER())
        - [`LOWER()`](#LOWER())
        - [`INITCAP()`](#LOWER())
        - [`LENGTH()`](#LENGTH())
        - [`TRIM()`](#TRIM())
        - [`SUBSTRING()`](#SUBSTRING())
        - [Concatenation Methods](#Concatenation)
        - [`REPLACE()`](#REPLACE())
        - [`COALESCE()`](#COALESCE())
    - **II. Conditionals**
        - [Boolean Statements](#Boolean-Statements)
        - [`CASE WHEN`](#CASE-WHEN)

    - **III. Date-Time Manipulation**
        - [Type Conversion](#Type-Conversion)
        - [`EXTRACT()`](#EXTRACT())
- [Intermediate SQL II: Subqueries](#Advanced-SQL-II:-Subqueries)

---

## The Dataset

We'll be using a dataset on BigQuery `hacker_news` that consists of all posts and comments since 2006 from the website [Hacker News](https://news.ycombinator.com) (HN). HN is a news website whose readership is primarily tech startup enthusiasts and coders.

> **WARNING:** Always use `LIMIT` when returning queries! This ensures you do not transfer large amounts of data. It is also a good idea to only retrieve the columns you need -- i.e. in general, do not use `*`.

The Hacker News website:
- Displays and ranks links to user-submitted stories:

![](./images/hacker-news.png)

- Allows users to comment on each story:
![](./images/hacker-news-comments.png)

> Note: If you want to do more with Hacker News data, there is no need to use Google. HN has an [officially sanctioned free API](https://blog.ycombinator.com/hacker-news-api/), a [reasonable scraping policy for most-recent posts](https://news.ycombinator.com/item?id=1721105), and there are numerous [data dumps online](https://www.kaggle.com/hacker-news/hacker-news).

Here are some inventive uses of the dataset: https://medium.com/google-cloud/big-data-stories-in-seconds-hacker-news-abe52bc5caad#.pjfjzr76j

---

## Review

__Query Order:__
1. SELECT
1. FROM
1. WHERE
1. GROUP BY 
1. HAVING
1. ORDER BY
1. LIMIT

__Mnemonic:__ "__S__melly __F__eet __W__ill __G__ive __H__orrible __O__dors, __L__ingeringly"

---
__Execution Order:__
1. FROM = get data
1. WHERE = filter rows
1. GROUP BY = aggregate
1. HAVING = filter aggregation
1. SELECT = display columns
1. ORDER BY = display sorted rows
1. LIMIT = display subset of rows

---

## Warmup -- Understanding the dataset

There are four tables inside `hacker_news`:
+ `comments`
+ `stories`
+ `full`
+ `full201510`

No documentation comes with this database (as is typical nearly everywhere).

So first, let's understand the relationship between the tables.

---

> **IMPORTANT NOTE:** Only use `COUNT`, `MIN`, and `MAX` in the `SELECT` clause for all answers below!
> - Feel free to answer using one query per table, if desired.
> - If your query is taking more than a few seconds, stop it and rewrite your query!
> - Observe how much data will be processed before running a query (right-middle in green) -- if it is more than normal, rewrite your query.
> - Always use `LIMIT` if your `SELECT` clause is not an aggregation.

#### 1. How many rows are in `comments`, `stories`, `full`, and `full_201510`?

_Write your answers down. You will need these numbers later._

```MySQL
-- code here
```

#### 2. When were the first and last rows in `comments`? `stories`? NOTE: TIMESTAMP_SECONDS(time) converts the INT64 `time` into a DATETIME.

```MySQL 
-- code here
```

#### 3. What is the likely difference between `full` and `full_201510`? Can you provide evidence? 

Probably `full_201510` is likely everything in `full` up to 10-2015.

```MySQL
-- code here
```

#### 4. Does `full` contain all of the rows in `comments`? `stories`? `full_201510`? (HINT: You may be able to use a JOIN.)

Yes.

```MySQL
-- code here

```

#### 5. Given what we learned above, do we have to use all four tables for our analysis? What might be some reasons why there are separate tables?

We only have to use `full` -- all other tables are included inside it. Why are there other tables?

Perhaps:
+ The dataset was originally up to 2015. They later changed it to make it continuously update.
+ Separating into comments and stories may make some queries run faster, since less data must be processed.

# Intermediate SQL I: Special Functions

Throughout this entire session, we'll be running the queries in Google BigQuery. This Jupyter Notebook will just be a written record of what we've learned so that you'll have all of these functions in one location.

Note that **THIS IS BY NO MEANS AN EXHAUSTIVE LIST** -- these contain some of the most common asked in interviews and/or useful on the job.

## I. String Manipulation

In this section, we will use the `stories` and `comments` tables to save data.

See the docs here: https://cloud.google.com/bigquery/docs/reference/standard-sql/string_functions

### `LOWER()`
This is the same as the `.lower()` method for strings in Python used to convert every letter in a string to lower case

_Example:_ Convert all letters of the string `HeLlO, wOrLd!` to lower case
```MySQL
SELECT LOWER('HeLlO, wOrLd!');
```

**EXERCISE 1:** Write a query that returns the lower-cased story titles of story ids < 100. (Remember to use `LIMIT`. Don't return all columns. Look at the size of data processed.)

_Answer:_

```MySQL
-- code here

```

### `UPPER()`
For completeness, this is the same as the `.upper()` method for strings in Python used to capitalize every letter in a string

_Example:_ Capitalize all letters of the string `Hello World`
```MySQL
SELECT UPPER('Hello, world!');
```

### `INITCAP()`
This is the same as the `.capitalize()` method for strings in Python that is used to convert the first letter to upper case.

**EXERCISE 2:** How many distinct Hacker News story authors uppercase the first letter of their username (in `stories`)? Note: `COUNT(DISTINCT <field>)` counts the number of distinct rows.

_Answer:_

```MySQL
-- code here

```

### `LENGTH()`
This is the same as the `len()` function in Python. However, since we don't have lists or tuples in SQL, this is only applicable to objects with characters.

_Example:_ Write a query that finds short usernames.

```MySQL
SELECT DISTINCT c.by
  FROM `bigquery-public-data.hacker_news.comments` c
  WHERE LENGTH(c.by) < 3;
```

### `TRIM()`
This is the same as the `.strip()` method for strings in Python that eliminates leading and trailing white spaces.

_Example:_ Write a query that strips out the white space from the string `'     Hello, world!     '`

```MySQL
SELECT TRIM('     Hello, world!     ');
```

### `SUBSTRING()`
Python doesn't have a function that extracts a substring since we can just do it by directly indexing through the string. If you're familiar with R though, then you'll recognize this is similar to the `substr()` function. The start position is 1-indexed!

Syntax for this function:

```MySQL
SELECT SUBSTRING(string_column, <start_position>, <length>);
```

**Example #1:**
```MySQL
SELECT SUBSTRING('Hello there, friend! Hehe.', 1, 5);
```
will return `'Hello'`

**Example #2:**
```MySQL
SELECT SUBSTRING('Hello there, friend! Hehe.', 14);
```
will return `'friend! Hehe.`

### Concatenation

This is the equivalent of string concatenation in Python using `+`. The `+` in Python is replaced by `||` in PostgreSQL. Alternatively, you can use the `CONCAT()` function.

_Example:_ Write a query that prints the username followed by a space followed by their story title.
```MySQL
SELECT author || ' ' || title
  FROM `bigquery-public-data.hacker_news.stories`
  LIMIT 100;
```

**EXERCISE 3:** Using `SUBSTRING` and `||`, can you display the usernames of comment posters with comment id < 100 with their first two letters capitalized?

_Answer:_

```MySQL
-- code here

```

### `REPLACE()`

This is the equivalent of the `.replace()` method for strings in Python and the `gsub()` function in R.

_Example:_
```MySQL
SELECT title, REPLACE(title, 'Woz', 'Steve Wozniak')
  FROM `bigquery-public-data.hacker_news.stories`
  WHERE id < 100
  ORDER BY id;
```

__Does the function work when replacing `NULL` values though?__

```MySQL
SELECT title, REPLACE(NULL, 'Woz', 'Steve Wozniak')
  FROM `bigquery-public-data.hacker_news.stories`
  WHERE id < 100
  ORDER BY id;
```

## `COALESCE()`
This is an extremely powerful function that lets us handle missing values on a column-by-column basis.

The syntax is pretty straight forward for this one: 
```MySQL
COALESCE(<column_name>, <fill_value>);
```

**EXERCISE 4**: Some stories do not have a URL -- let's point them to GA! Can you display the author, URL, and title of 100 stories where a `NULL` URL is replaced with `https://ga.co`?

_Answer:_

```MySQL
-- code here
```

_____
## II. Conditionals

### Boolean Statements

We can also include Booleans to create dummy variables in SQL on the fly.

_Example:_
```MySQL
SELECT LENGTH(text) < 100 AS `is_short`, text
  FROM `bigquery-public-data.hacker_news.comments`
  WHERE id < 100
  LIMIT 100;
```

## `CASE WHEN`
This is the equivalent of if-elif-else statements, except embedded into a query. This takes Boolean Statements to the next level by allowing you to customize what happens on a case-by-case basis

_Example:_ Write a query that groups bannermen army sizes into 'large' (35+), 'medium' (25-34), 'small' (< 25) 

```MySQL
SELECT author, score,
       CASE WHEN score >= 15 THEN 'highly regarded'    -- if
            WHEN score >= 10 THEN 'noticed'            -- elif
            ELSE 'insignificant'                       -- else
       END AS story_importance                         -- end it! (and rename if you want)
FROM `bigquery-public-data.hacker_news.stories`
ORDER BY id DESC
LIMIT 1000;
```

`CASE WHEN` is often used in conjunction with a `GROUP BY` to bin results. 

> Unfortunately, you must repeat the `CASE` to display the categories in `SELECT`:

_Example:_ 
```MySQL
SELECT CASE WHEN score >= 100 THEN 'highly regarded'
            WHEN score >= 50 THEN 'noticed' 
            ELSE 'insignificant'
       END AS story_importance,
  COUNT(*)
FROM `bigquery-public-data.hacker_news.stories`
GROUP BY story_importance
LIMIT 1000;
```

NOTE: In most SQL variants, `GROUP BY` is evaluated before `SELECT`. So, you must repeat the `CASE` inside the `GROUP BY`. Luckily, BigQuery saves us from this repetition!

## III. Date-Time Manipulation

See the docs here: https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions

### Type Conversion

#### `DATETIME()`
You can create a DATETIME from scratch:
```MySQL
-- Note the times are always stored internally as UTC (example from the docs)
SELECT
  DATETIME(2008, 12, 25, 05, 30, 00) as datetime_ymdhms,
  DATETIME(TIMESTAMP "2008-12-25 05:30:00+00", "America/Los_Angeles") as datetime_tstz;
```

#### `DATETIME()` with `TIMESTAMP_SECONDS()`

You can convert an timestamp type into a DATETIME by first converting to a timestamp (`TIMESTAMP_SECONDS()`) then to a datetime (`DATETIME()`).

```MySQL
SELECT DATETIME(TIMESTAMP_SECONDS(time))
  FROM `bigquery-public-data.hacker_news.stories`
  LIMIT 100;
```

#### `CURRENT_DATETIME()`
You can use this to pull the current date (as a datetime type) from your computer's clock
```MySQL
SELECT CURRENT_DATETIME();
```

Or just the date (as a date type):
```MySQL
SELECT CURRENT_DATE();
```

#### `FORMAT_DATETIME`

SQL is incredibly flexible in how you can display dates and times. 

_Example:_
```MySQL
SELECT FORMAT_DATETIME("%b %Y", DATETIME(TIMESTAMP_SECONDS(time))),
       title
  FROM `bigquery-public-data.hacker_news.stories`
  LIMIT 1000;
```

Here are all of the possible options: https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions#supported_format_elements_for_datetime

**EXERCISE 5:** Write a query that returns what the date was 21 days ago. Format it in a nice way! (Hint: Perhaps one of the other DateTime functions can help: https://cloud.google.com/bigquery/docs/reference/standard-sql/datetime_functions#datetime_sub)

_Answer:_

```MySQL
-- code here
```

### `EXTRACT()`

If you want to extract certain parts of a datetime object, this function is MAGICAL!

```MySQL
SELECT CURRENT_DATETIME() AS `Today`,
	   EXTRACT(DAY FROM CURRENT_DATETIME()) AS `Day`,
	   EXTRACT(MONTH FROM CURRENT_DATETIME()) AS `Month`,
	   EXTRACT(YEAR FROM CURRENT_DATETIME()) AS `Year`,
	   EXTRACT(HOUR FROM CURRENT_DATETIME()) AS `Hour`,
	   EXTRACT(MINUTE FROM CURRENT_DATETIME()) AS `Minute`
```

### Challenge: Interview Questions

Now let's use the `full` table to try to write real-time queries! 
- We want to run these at any time to collate the latest results. 
- All of the below require `GROUP BY`.

> Note: Instead of `author`, you'll now have to use `by` in conjunction with `type`.

**EXERCISE 6: When was the last item of each `type` posted? Make sure the date and time are human-readable!**

_Answer:_

```MySQL
-- code here
```

**EXERCISE 7: Which hour of the day is most popular for posting a story?**

```MySQL
-- code here
```

**EXERCISE (BONUS): Who are the top ten submitters of stories in the last 7 days?**

_Answer:_

```MySQL
-- code here
-- Assume NULL is okay
```

***
# Intermediate SQL II: Subqueries

## What is a subquery?

Exactly what it sounds like: **it's a query within a query**!

...What?! Sounds complicated...why do we need this?

**Motivation:** How can we get _all_ stories that are tied for highest score?

> Note we cannot just use ORDER BY, since we must know in advance what the LIMIT should be!

_Answer:_

```MySQL
-- code here
SELECT f.by, f.score, f.title
  FROM `bigquery-public-data.hacker_news.full` f
  WHERE f.score = 
    (SELECT MAX(score)
     FROM `bigquery-public-data.hacker_news.full`);
```

### Subqueries in `WHERE`

How did we think about this?
- __The intention of a subquery is to utilize a a list (column) or single value.__
- If we choose to extract just one column from a table using a query, we essentially have a list
- We've written WHERE statements before with `IN` and `NOT IN` and compared results to a list
- Connecting the dots: we can replace the list in a WHERE clause with a subquery to make things more dynamic

**Exercise 8:** How many users have posted a story but never commented?

> First, we have to find the user IDs who have commented. Then, we see who has posted a story but is `NOT IN` these IDs.

_Answer:_

```MySQL
-- code here
```

_**Short Note on Efficient Queries**_

Some `JOIN` commands (especially `INNER JOIN`) can be very computationally intensive. This is why sometimes we would prefer to write subqueries.

_Example:_ What are the 502 comments that are in `full_201510` but not in `stories`? (We saw this in the warmup!)

```MySQL
SELECT f.id, f.by, f.title
  FROM `bigquery-public-data.hacker_news.full_201510` f
  WHERE f.type = 'comment' AND 
        f.id NOT IN 
          (SELECT id FROM`bigquery-public-data.hacker_news.comments`);
```

### Subqueries in `SELECT`

Subqueries can show up almost anywhere in the query! If we want to compare values to __a single value,__ we could include the result of a subquery in the `SELECT` clause. This is especially important when you want to construct some sort of **_benchmark_** (e.g. how much you have missed/beaten a sales target by, what the active returns of a mutual fund is compared to its benchmark index, etc.)  

_Example:_ How do we determine what percentage of stories do not have a URL?

```MySQL
SELECT COUNT(*) / (SELECT COUNT(*) FROM `bigquery-public-data.hacker_news.stories`)
  FROM `bigquery-public-data.hacker_news.stories`
  WHERE url IS NULL;
```

_Example:_ How do we show the average score next to each story?
```MySQL
SELECT title, score, (SELECT AVG(score) FROM `bigquery-public-data.hacker_news.stories`)
  FROM `bigquery-public-data.hacker_news.stories`
  LIMIT 100;
```

#### _Short Note on Order of Execution in SQL Queries_
Across clauses, there is a sequence that queries follow. SQL queries will run `FROM` first, then `WHERE` and other filters, and then `SELECT` last. So in the exercise **below**, the `highest_salary` is already going to be calculated based on Asia and Canada employees because `WHERE` executes before `SELECT`.

However, within a clause (e.g. within SELECT) everything runs ___simultaneously___, not sequentially! So you cannot use `highest_salary` in say a calculation for "difference" -- you will need to use the actual subquery in the calculation.

## Interview Challenges

### Challenge Interview Question \#1

Which user has the largest number of dead posts? (You can assume there is no tie. The user must be non-`NULL`.)

_Answer:_ 

```MySQL
-- code here
```

### Challenge Interview Question #2

What percentage of stories get at least one comment?

> The challenge here is understanding how stories are linked to comments!

_Answer:_

```MySQL
-- code here

```

### Challenge Interview Question \#3

Which stories were posted for the longest amount of time before getting a comment?

> The challenge here is thinking carefully about the time data types.

_Answer:_ 

```MySQL
-- code here
```

## Next Steps
- [Hacker Rank](https://www.hackerrank.com/domains/sql)
- [Code Wars](https://www.codewars.com/)
- [SQL Zoo](https://sqlzoo.net/)
- [Code Academy](https://www.codecademy.com/learn/learn-sql)
- [mystery.knightlab](https://mystery.knightlab.com/)
- [tutorialspoint](https://www.tutorialspoint.com/sql/index.htm)
- [mode.com](https://mode.com/sql-tutorial/)
- [sqlbolt.com](https://sqlbolt.com/lesson/select_queries_introduction)
- [w3resource](https://www.w3resource.com/sql-exercises/)
- [w3schools](https://www.w3schools.com/sql/sql_exercises.asp)
- [techbeamers](https://www.techbeamers.com/sql-query-questions-answers-for-practice/)
- [mystery.knightlab](http://mystery.knightlab.com/)
- [pgexercises.com](https://pgexercises.com/)
- [jitbit](https://www.jitbit.com/news/181-jitbits-sql-interview-questions/)
- [quip.com](https://quip.com/2gwZArKuWk7W)
- [plateau-workshop](https://plateau-workshop.org/assets/papers-2019/10.pdf)